In [496]:
import openai as oai
import os
import pandas as pd
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import requests
from typing import List

In [497]:
def get_songlist(playlist_orig_name: str) -> List[str]:
    scope = "playlist-read-private"
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
    playlists = sp.current_user_playlists()
    songlist = []
    for playlist in playlists['items']:
        if playlist['name'] == playlist_orig_name:
            results = sp.playlist_tracks(playlist['id'])
            for idx, item in enumerate(results['items']):
                track = item['track']
                song = track['artists'][0]['name'] + " – " + track['name']
                songlist.append(song)
    return songlist

In [537]:
def create_chat_prompt(songlist: List[str]) -> str:
    chat_prompt = "Make me a new 5 song spotify playlist, consisting of songs officially released (no unreleased music). "
    chat_prompt += "Each song recommendation should be based on these 5 songs, and if possible, based on more than 1 song at a time. "
    chat_prompt += "Each song should only print the main artist, and the title should be the officially released title."
    chat_prompt += "The reasoning for choosing each song should be at least a sentence, meaningful, and explain why it was recommended. : \n"
    for song in songlist:
        chat_prompt += song + "\n"
    chat_prompt += "The exact formatting should be: \n 'Artist' - 'Song Name' - 'Reasoning'. Don't use ' - ' in your reasoning."
    return chat_prompt


In [538]:
def get_recommendations(chat_prompt: str, openAIcount: int) -> str:
    oai_response = oai.Completion.create(
        model="text-davinci-003", prompt=chat_prompt, temperature=1, max_tokens=1000)
    new_playlist_response = oai_response['choices'][0]['text'].strip()
    openAIcount += 1
    print("OpenAI calls: " + str(openAIcount))
    return new_playlist_response

In [539]:
def recomendations_to_list(new_playlist_response: str) -> List[str]:
    songs = []
    new_playlist_songs = list(filter(None, new_playlist_response.split('\n')))
    for song in new_playlist_songs:
        artist, title, reasoning = song.split(' - ')
        song_dict = {"title": title, "artist": artist, "reasoning": reasoning}
        songs.append(song_dict)
    return songs

In [540]:
def get_track_ids(songs: List[str]) -> List[str]:
    scope = "playlist-read-private"
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
    track_ids = []
    for song in songs:
        artist = song['artist']
        title = song['title']
        result = sp.search(q="track:{} artist:{}".format(
            title, artist), type='track')
        if result['tracks']['items']:
            track_ids.append(result['tracks']['items'][0]['id'])
        else:
            print('Could not find track ID for {} by {}'.format(title, artist))
            track_ids.append('N/A')
    return track_ids

In [541]:
def create_hastebin(new_playlist_response: str, hastebin_token: str) -> str:
    url = "https://hastebin.com/documents"
    response = requests.post(url, headers={"Authorization": f"Bearer {hastebin_token}",
                             "content-type": "text/plain"}, data=new_playlist_response.encode('utf-8'))
    if response.status_code == 200:
        key = response.json()["key"]
        haste_url = f"https://hastebin.com/share/{key}"
    else:
        haste_url = "Failed to create hastebin"
    return haste_url

In [542]:
def create_playlist(track_ids: List[str], haste_url: str) -> str:
    playlist_name = "spotAI generated"
    playlist_description = f'A playlist generated by spotAI. Learn more about the selection of each song at: {haste_url}'
    scope = 'playlist-modify-public'

    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

    new_playlist = sp.user_playlist_create(user=sp.current_user(
    )['id'], name=playlist_name, description=playlist_description)
    playlist_id = new_playlist['id']
    track_ids = [id for id in track_ids if id != 'N/A']

    sp.playlist_add_items(playlist_id=playlist_id, items=track_ids)

    for i in range(0,10):
        sp.playlist_change_details(
            playlist_id=playlist_id, description=playlist_description)

    return 'complete'


In [543]:
def main():
    load_dotenv()
    oai.api_key = os.getenv("OPENAI_API_KEY")
    hastebin_token = os.getenv("HASTEBIN_TOKEN")
    playlist_orig_name = "spotAI"
    openAIcount = 0
    songlist = get_songlist(playlist_orig_name)
    chat_prompt = create_chat_prompt(songlist)
    new_playlist_response = get_recommendations(chat_prompt, openAIcount)
    songs = recomendations_to_list(new_playlist_response)
    track_ids = get_track_ids(songs)
    # while (track_ids.count('N/A') >= 2 & openAIcount < 3):
    #     openAIcount += 1
    #     new_playlist_response = get_recommendations(chat_prompt)
    #     songs = recomendations_to_list(new_playlist_response)
    #     track_ids = get_track_ids(songs)
    haste_url = create_hastebin(new_playlist_response, hastebin_token)
    create_playlist(track_ids, haste_url)


In [544]:
if __name__ == "__main__":
    main()

Using `localhost` as redirect URI without a port. Specify a port (e.g. `localhost:8080`) to allow automatic retrieval of authentication code instead of having to copy and paste the URL your browser is redirected to.


OpenAI calls: 1


Using `localhost` as redirect URI without a port. Specify a port (e.g. `localhost:8080`) to allow automatic retrieval of authentication code instead of having to copy and paste the URL your browser is redirected to.
